In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
import gc

In [7]:
df = pd.read_csv("../andrews_dataset.dat", dtype=np.float16)

In [8]:
df

,Tempo_bonus,Shielding pawn missing,Shielding pawn missing.1,Shielding pawn missing.2,Shielding pawn missing.3,Shielding pawn on open missing,Shielding pawn on open missing.1,Shielding pawn on open missing.2,Shielding pawn on open missing.3,Pawn doubled,...,Psqt.376,Psqt.377,Psqt.378,Psqt.379,Psqt.380,Psqt.381,Psqt.382,Psqt.383,Phase,Label
0,1.0,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,128.000000,0.5
1,1.0,0.0,-1.0,0.0,1.0,-1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.570312,1.0
2,-1.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.5
4,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.078125,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155536,-1.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0
155537,-1.0,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.812500,1.0
155538,1.0,0.0,0.0,1.0,-1.0,0.0,0.0,1.0,-1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.085938,0.5
155539,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,119.250000,0.5


In [ ]:
import gc
gc.collect()
df = df.sample(frac=1).reset_index(drop=True)

In [9]:
arr = np.array(df)

In [ ]:
del df
import gc
gc.collect()

In [ ]:
df = pd.read_csv("../training.dat")
arr = np.array(df)
arr.shape

In [4]:
X = df.iloc[:, :676]
Y = df.iloc[:, -1]
N = len(X)
train_test_split = 0.95
N_X = int(train_test_split * N)
x_train = X[:N_X]
y_train = Y[:N_X]
x_test = X[N_X:]
y_test = Y[N_X:]

In [10]:
X = arr[:, :676]
Y = arr[:, -1]
order = np.arange(len(X))
np.random.shuffle(order)
X = X[order]
Y = Y[order]
N = len(X)
train_test_split = 0.8
N_X = int(train_test_split * N)
x_train = X[:N_X]
y_train = Y[:N_X]
x_test = X[N_X:]
y_test = Y[N_X:]

In [11]:
inputs = tf.keras.Input(shape=(676,))
x = tf.keras.layers.Dense(8, activation = tf.nn.relu)(inputs)
#x = tf.keras.layers.Dense(32, activation = tf.nn.relu)(x)
outputs = tf.keras.layers.Dense(1, activation = tf.nn.sigmoid)(x)
model = tf.keras.Model(inputs = inputs, outputs = outputs)

In [12]:
model.compile(loss=tf.keras.losses.MSE,metrics=["accuracy"] )

In [ ]:
def write_weights(model, file):
    with open(file, 'w') as f:
        weights = model.get_weights()
        for i in range(len(weights)):
            w = weights[i]
            if i % 2 == 0:
                layer_desc = "WEIGHTS"
                layer_type = "[f32;{}]".format(w.shape[0] * w.shape[1])
                s = "pub const LAYER_{}_WEIGHTS_SHAPE: (usize, usize) = ({},{});\n".format(i//2, w.shape[0],w.shape[1])
            else:
                layer_desc = "BIAS"
                layer_type = "[f32;{}]".format(len(w))
                s = ""

            s += "#[rustfmt::skip]\npub const LAYER_{}_{} : {} = ".format(i//2,layer_desc,layer_type)
            s += "[";
            for j in range(len(w)):
                if i % 2 == 0:
                    for k in range(len(w[j])):
                        s += str(w[j,k])
                        if k != len(w[j]) - 1:
                            s += ","
                else:
                    s += str(w[j])
                s += ","
            s += "];\n"
            f.write(s)

In [ ]:
import math
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [14]:
save = "train/model7"
import gc
import time
for i in range(100):
    gc.collect()
    time.sleep(5)
    #model.fit(x_train[:2_500_000], y_train[:2_500_000], epochs = 1)
    #model.fit(x_train[2_500_000:], y_train[2_500_000:], epochs = 1)
    #model.evaluate(x_test,y_test)
    model.fit(x_train,y_train,validation_data=(x_test,y_test), epochs=1)
    model.save(save+"_"+str(i)+".h5")
#write_weights(model, save+".weights")

3402/3889 [=========================>....] - ETA: 0s - loss: 0.0983 - accuracy: 0.3804

KeyboardInterrupt: 

In [ ]:
write_weights(model, save+".weights")

In [ ]:
other_model = tf.keras.models.load_model("train/model4_7.h5")
other_model.evaluate(x_test, y_test)

In [ ]:
test = np.zeros((1, 676), dtype=np.float32)
other_model.predict(test)